# Lesson 1: Model, Prompts and Parser
This notebook documents my learning journey on **LangChain for LLM Application Development** course from Deeplearning.ai \
[Lesson 1: Model, Prompts and Parser](https://learn.deeplearning.ai/courses/langchain/lesson/xf7wh/models,-prompts-and-parsers).

\
What I Learned
- How to set up the environment: Installing necessary libraries like `python-dotenv` and `langchain-groq`, and managing API keys securely.
- How to interact with LLMs: Using `init_chat_model` to connect to a language model and send prompts.
- How to use Prompt Templates: Creating dynamic and reusable prompts with `ChatPromptTemplate` to guide the LLM's output.
- How to structure LLM output: Using `StructuredOutputParser` and `ResponseSchema` to get structured JSON data from the LLM.
- How to chain components with LCEL: Using the pipe operator (`|`) to create a clean and readable pipeline of operations.

## I. Setting up the Environment

First, we need to install a few libraries:

In [ ]:
!pip install -qU python-dotenv
!pip install -qU langchain-groq

In [ ]:
!pip show langchain langchain-groq

Name: langchain
Version: 0.3.27
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: 
---
Name: langchain-groq
Version: 0.3.6
Summary: An integration package connecting Groq and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: groq, langchain-core
Required-by: 


create `.env` file and
replace `secret-api-key` with your actual credentials.
```
GROQ_API_KEY=secret-api-key
```

Loads environment variables from the .env file

In [ ]:
import os
from dotenv import load_dotenv
_ = load_dotenv(override=True) # read local .env file

## II. Interacting with LLMs

Now that we have our environment set up, let's start interacting with a language model. We'll use the function `init_chat_model()` from `langchain.chat_models` to connect to a specific LLM model. In this case, I'm using `llama-3.3-70b-versatile`

First, we need to set some model configurations.

In [ ]:
# configs
configs = {
    "model": "llama-3.3-70b-versatile",
    "model_provider": "groq",
    "temperature": 0
}

Initializes the chat model and sends a simple message to see a response.

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage

# Initialize the model
llm = init_chat_model(**configs)

# Simple chat interaction
messages = [HumanMessage(content="What is 1 + 1?")] # remember: messages must be a list
response = llm.invoke(messages)
print(response.content)

1 + 1 = 2.


In [ ]:
# just see what type the chat model is
print(type(llm))

<class 'langchain_groq.chat_models.ChatGroq'>


## III. Prompt template
Prompt Templates in LangChain allow you to create flexible, reusable prompts for interacting with language models. You define a structure with placeholders and fill them with specific information each time.

### Create a template with placeholders

We use `{}` to create placeholders in the prompt template.

In [ ]:
template_string = """Translate the text \
that is delimited by the keyword "text" \
into a style specified by the keyword "style"

style: {style}

text: {text}

provide the translated sentence directly as the response
"""

We use `ChatPromptTemplate.from_template()` to create a prompt template from a string

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In LangChain, the `prompt_template.messages` is a list because a prompt can include multiple messages, allowing for a flexible, multi-step conversation structure. Each message in the list represents a different part of the interaction (e.g., user input or model response).  \
Since there's only one message, we use `[0]` to access it directly.

In [ ]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by the keyword "text" into a style specified by the keyword "style"\n\nstyle: {style}\n\ntext: {text}\n\nprovide the translated sentence directly as the response\n')

In [ ]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

### Fill in the placeholders

Define values for `style` and `text` placeholders, like `customer_style` and `customer_email`.


In [ ]:
customer_style = "a calm and respectful tone in American English"

In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

We use `format_messages()` to fill in the placeholders of our `prompt_template` with the defined values (`customer_style` and `customer_email`). This returns a list of formatted messages, and we can then inspect the type of `customer_messages` and access the first message using `[0]`.

In [ ]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [ ]:
# Just see what type it is
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [ ]:
# Just see the content that we will pass to the model
print(customer_messages[0].content)

Translate the text that is delimited by the keyword "text" into a style specified by the keyword "style"

style: a calm and respectful tone in American English

text: 
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!


provide the translated sentence directly as the response



### Pass the prompt to the LLM

Calls the language model to generate a response matching the customer's tone and style, then prints the output.

In [ ]:
customer_response = llm.invoke(customer_messages)

In [ ]:
print(customer_response.content)

I'm extremely frustrated that my blender lid came off and splattered my kitchen walls with smoothie, and to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen, so I would greatly appreciate your assistance with this issue as soon as possible.


This is another example showing the reusability of the prompt template. We reuse the same template to format a different message (`service_reply`), but this time with a **Pirate English** style (`service_style_pirate`). This demonstrates how a single template can be adapted for various inputs and styles.

In [ ]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [ ]:
service_style_pirate = "a polite tone that speaks in English Pirate"

In [ ]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by the keyword "text" into a style specified by the keyword "style"

style: a polite tone that speaks in English Pirate

text: Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!


provide the translated sentence directly as the response



In [ ]:
service_response = llm.invoke(service_messages)

In [ ]:
print(service_response.content)

Arrr, greetings to ye, valued customer! I be sorry to inform ye that the warranty don't be coverin' the cleanin' expenses for yer kitchen, matey. It seems ye had a bit of a mishap with yer blender, forgettin' to put the lid on before setlin' her to work, savvy? Unfortunately, that be considered misuse, and we can't be held responsible for the mess. Better luck next time, me hearty! May the winds o' fortune blow in yer favor, and may ye have a grand day, nonetheless! Fair winds!


## IV. Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [ ]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

### Prompt Setup

In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [ ]:
messages = prompt_template.format_messages(text=customer_review)
response = llm.invoke(messages)
print(response.content)

```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [ ]:
type(response.content)

str

In [ ]:
# You will get an error by running this line of code
# because'gift' is not a dictionary
# 'gift' is a string

# DON'T WORRY ABOUT THIS ERROR
# response.content.get('gift')

### Parse the LLM output string into a Python dictionary

In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [ ]:
gift_schema = ResponseSchema(
    name="gift",
    description="Was the item purchased as a gift for someone else? "
                "Answer true if yes, false if not or unknown. "
                "Output data type: Boolean."
)

delivery_days_schema = ResponseSchema(
    name="delivery_days",
    description="How many days did it take for the product to arrive? "
                "If this information is not found, output -1. "
                "Output data type: Integer."
)

price_value_schema = ResponseSchema(
    name="price_value",
    description="Extract any sentences about the value or price, "
                "and output them as a comma-separated Python list. "
                "Output data type: List of strings."
)

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]


In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
format_instructions = output_parser.get_format_instructions()

In [ ]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else? Answer true if yes, false if not or unknown. Output data type: Boolean.
	"delivery_days": string  // How many days did it take for the product to arrive? If this information is not found, output -1. Output data type: Integer.
	"price_value": string  // Extract any sentences about the value or price, and output them as a comma-separated Python list. Output data type: List of strings.
}
```


In [ ]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, \
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(
    text=customer_review,
    format_instructions=format_instructions
)

In [ ]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "`

In [ ]:
response = llm.invoke(messages)

In [ ]:
print(response.content)

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [ ]:
output_dict = output_parser.parse(response.content)

In [ ]:
output_dict

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [ ]:
# Congratulations! You now have a Python dict instead of str.
# This is the power of LangChain's output parser.

type(output_dict)

dict

In [ ]:
output_dict.get('delivery_days')

2

## Extra: Put all together with LCEL (LangChain Expression Language)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [ ]:
configs = {
    "model": "llama-3.3-70b-versatile",
    "model_provider": "groq",
    "temperature": 0
}

llm = init_chat_model(**configs)

In [ ]:
prompt_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, \
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=prompt_template)

In [ ]:
gift_schema = ResponseSchema(
    name="gift",
    description="Was the item purchased as a gift for someone else? "
                "Answer true if yes, false if not or unknown. "
                "Output data type: Boolean."
)

delivery_days_schema = ResponseSchema(
    name="delivery_days",
    description="How many days did it take for the product to arrive? "
                "If this information is not found, output -1. "
                "Output data type: Integer."
)

price_value_schema = ResponseSchema(
    name="price_value",
    description="Extract any sentences about the value or price, "
                "and output them as a comma-separated Python list. "
                "Output data type: List of strings."
)

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
partial_prompt = prompt.partial(format_instructions=output_parser.get_format_instructions)

In [ ]:
chain = partial_prompt | llm | output_parser

In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

In [ ]:
response = chain.invoke({"text": customer_review})

response

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [ ]:
print(type(response))

<class 'dict'>


* * *
# Summary of Your LangChain Learning Journey

In this notebook, you've covered the fundamental building blocks of creating powerful applications with LangChain. Here's a breakdown of the key concepts:

**1. Setting Up Your Environment:**

* You started by installing the necessary libraries, including:
- `python-dotenv` to manage your API keys securely
- `langchain-groq` to initialize the ChatGroq class with init_chat_model
* You learned how to load your API keys from a `.env` file, which is a best practice for keeping your credentials safe and separate from your code.

**2. Interacting with Language Models (LLMs):**

* You instantiated a chat model from `init_chat_model()`.
* You sent a simple message to the LLM and received a response, demonstrating the basic request-response cycle.

**3. Crafting Precise Prompts with Prompt Templates:**

* You discovered the power of `ChatPromptTemplate` to create reusable and dynamic prompts. This allows you to define a template with placeholders (like `{style}` and `{text}`) and then fill them in with specific values.
* You saw how to format the prompt template with different styles and text, creating tailored messages to send to the LLM. This is crucial for guiding the model's output and achieving your desired results.

**4. Structuring and Parsing LLM Outputs:**

* You encountered a common challenge: the LLM's output is often a raw string, which can be difficult to work with programmatically.
* You learned how to use `StructuredOutputParser` and `ResponseSchema` to define a specific JSON structure for the LLM's response.
* By providing these format instructions in your prompt, you guided the LLM to generate a well-formed JSON object.
* Finally, you used the `output_parser` to parse the LLM's string response into a Python dictionary, making it easy to access and use the extracted information.

**5. Chaining it all together with LangChain Expression Language (LCEL):**

*   You learned how to use the pipe symbol (`|`) to chain together the prompt, the model, and the output parser into a single, elegant pipeline. This makes your code more concise and readable.

**In essence, you've learned how to:**

* **Connect** to a powerful language model.
* **Communicate** your instructions effectively using prompt templates.
* **Control** the output format to get structured, usable data.
* **Chain** these components together to create a streamlined workflow.

These are the foundational skills you'll need to build more complex and sophisticated LangChain applications. Keep up the great work!

* * *
# Key Commands and Imports to Remember

### Python Libraries:
- **`import os`**: Interacts with the operating system, mainly for accessing environment variables.
- **`from dotenv import load_dotenv`**: Loads environment variables from a `.env` file to securely manage API credentials.

### LangChain Libraries:
- **`from langchain.chat_models import init_chat_model`**: Easily initializes a chat model instance.
- **`from langchain_core.messages import HumanMessage`**: Represents user messages in the chat interaction.
- **`from langchain_core.prompts import ChatPromptTemplate`**: For creating and formatting prompt templates.
- **`from langchain.output_parsers import ResponseSchema, StructuredOutputParser`**: For parsing and structuring LLM outputs.

### Key LangChain Classes and Functions:
- **`init_chat_model()`**: Easily initializes a chat model instance.
- **`HumanMessage()`**: Used to create user messages in the chat interaction.
- **`ResponseSchema()`**: Defines a schema for expected responses, including name and description.  It's good practice to specify the output data type in the description.
- **`ChatPromptTemplate.from_template()`**: Convenient method to create a prompt template from a string.
- **`prompt_template.format_messages()`**: Formats the prompt template with specific values (e.g., style, text).
- **`prompt.partial()`**: A method to partially format a prompt template, which is useful when you want to pre-fill some of the variables.
- **`StructuredOutputParser.from_response_schemas()`**: Creates an output parser based on response schemas.
- **`output_parser.get_format_instructions()`**: Generates formatting instructions for the LLM output.
- **`output_parser.parse()`**: Parses the LLM output (string) into a Python dictionary.